In [299]:
import numpy as np
import scipy

In [300]:
R1, R2 = np.eye(3), np.eye(3)
t1 = np.asarray([0, 0, 1]).reshape(3, 1)
t2 = np.asarray([0, 0, 20]).reshape(3, 1)
K1, K2 = np.asarray([[700, 0, 600], [0, 700, 400], [0, 0, 1]]), np.asarray(
    [[700, 0, 600], [0, 700, 400], [0, 0, 1]]
)
Q = np.asarray([1, 1, 0]).reshape(3, 1)

In [301]:
def Pi(x: np.array):
    """
    Converts homogeneous to inhomogeneous coordinates
    Args:
        x (np.array) homogeneous coordinate

    Return:
        np.array converted inhomogeneous coordinate
    """

    return x[:-1] / x[-1]


def Piinv(x: np.array):
    """
    Converts inhomogeneous to homogeneous coordinates

    Args:
        x (np.array) inhomogeneous coordinate

    Return:
        np.array converted homogeneous coordinate
    """
    if x.ndim == 1:
        return np.concatenate((x, np.ones(1)))
    return np.vstack((x, np.ones((1, x.shape[1]))))

In [302]:
def projectpoints(K: np.array, R: np.array, t: np.array, Q: np.array):
    """
    Obtains projected 2D coordinates from world coordinates

    Args:
        K (np.array): intrinsics matrix
        R (np.array): extrinsic rotation matrix
        t (np.array): extrinsic translation matrix
        Q (np.array): homogeneous input points in world coordinates

    Return:
        projected_points (np.array): projected 2D points in homogeneous coordinates
    """
    # Projection matrix = K[R t] Q
    if Q.shape[0] == 3:
        Q = Piinv(Q)
    extrinsics = np.concatenate((R, t), axis=1)
    projected_points = K @ extrinsics @ Q
    return projected_points

## Ex 5.1

In [303]:
extrinsics1 = np.concatenate((R1, t1), axis=1)
P1 = K1 @ extrinsics1
extrinsics2 = np.concatenate((R2, t2), axis=1)
P2 = K2 @ extrinsics2
q1 = Pi(projectpoints(K1, R1, t1, Q))
q2 = Pi(projectpoints(K2, R2, t2, Q))
q1, q2

(array([[1300.],
        [1100.]]),
 array([[635.],
        [435.]]))

## Ex 5.2

In [304]:
q1_noise = q1 + np.asarray([1, -1]).reshape(2, 1)
q2_noise = q2 + np.asarray([1, -1]).reshape(2, 1)

In [305]:
def triangulate(pixel_coords: np.array, proj_matrices: np.array):
    """
    Given a list of pixel coordinates and projection matrices, triangulate to a common 3D point
    Args:
        pixel_coords (np.array): list of pixel coordinates in inhomogeneous coordinates
        proj_matrices (np.array): list of projection matrices

    Return:
        triangle (np.array): triangulated 3D point
    """
    n = pixel_coords.shape[0]
    # B_stack = []
    B_stack = np.zeros((n * 2, 4))
    for i in range(n):
        x, y = pixel_coords[i]
        proj_matrix = proj_matrices[i]
        B = np.asarray(
            [
                proj_matrix[2, :] * x - proj_matrix[0, :],
                proj_matrix[2, :] * y - proj_matrix[1, :],
            ]
        )
        B_stack = np.vstack((B_stack, B))
    U, S, Vt = np.linalg.svd(B_stack)
    # Get the smallest vector
    # print(f"Vt is {Vt}")
    triangle = Vt[-1, :]
    triangle /= triangle[-1]
    return triangle

In [306]:
pixel_coords = np.array([q1_noise, q2_noise])
proj_matrices = np.array([P1, P2])
Q_estimated = triangulate(pixel_coords, proj_matrices).reshape(4, 1)
pixel_coords.shape, proj_matrices.shape, Q_estimated.shape

((2, 2, 1), (2, 3, 4), (4, 1))

In [317]:
q1_reprojected = Pi(P1 @ Q_estimated)
q2_reprojected = Pi(P2 @ Q_estimated)
print(q1, q1_reprojected)
print(q2, q2_reprojected)
diff1 = np.linalg.norm(q1_reprojected - q1_noise)
diff2 = np.linalg.norm(q2_reprojected - q2_noise)
diff1, diff2, np.linalg.norm(Pi(Q_estimated) - Q)

[[1300.]
 [1100.]] [[1310.48950027]
 [1089.4923513 ]]
[[635.]
 [435.]] [[635.53411968]
 [434.4839772 ]]


(13.433018988192021, 0.6717725840473774, 0.021221817353381443)

We expect the linear algorithm to place a larger weight on the error of camera 2 than camera 1,
as it has a larger s. Therefore camera 2 having the smallest reprojection error is as we expected.
∥∥∥Q − ˜Q∥∥∥2 = 0.021

## Ex 5.3

https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html

funcallable
Function which computes the vector of residuals, with the signature fun(x, *args, **kwargs), i.e., the minimization proceeds with respect to its first argument. The argument x passed to this function is an ndarray of shape (n,) (never a scalar, even for n=1). It must allocate and return a 1-D array_like of shape (m,) or a scalar. If the argument x is complex or the function fun returns complex residuals, it must be wrapped in a real function of real arguments, as shown at the end of the Examples section.

In [308]:
import scipy.optimize


def triangulate_nonlin(pixel_coords: np.array, proj_matrices: np.array):
    """
    Given a list of pixel coordinates and projection matrices, triangulate to a common 3D point using a non linear approach
    Args:
        pixel_coords (np.array): list of pixel coordinates in inhomogeneous coordinates
        proj_matrices (np.array): list of projection matrices

    Return:
        triangle (np.array): triangulated 3D point
    """
    def compute_residuals(Q: np.array):
        """
        Compute residuals between projected points and observed pixel coordinates.
        Args:
            Q (np.array): Current estimate of 3D point (x, y, z) in homogeneous coordinates

        Return:
            residuals (np.array): Vector of residuals (differences between projected and observed points)
        """
        if Q.shape[0] == 3:
            Q = Piinv(Q)
        residuals = np.zeros(2 * len(pixel_coords))
        for i, q in enumerate(pixel_coords):
            projected_point = Pi(proj_matrices[i, :, :] @ Q)
            diff_vector = q.reshape(-1) - projected_point.reshape(-1)
            residuals[2 * i : 2 * (i + 1)] = diff_vector
        return residuals

    # Initial guess
    x0 = triangulate(pixel_coords, proj_matrices).reshape(-1)
    least_error_3D = scipy.optimize.least_squares(compute_residuals, x0)
    return least_error_3D.x

In [309]:
optimised_Q = triangulate_nonlin(pixel_coords, proj_matrices).reshape(4, 1)
Pi(optimised_Q)

array([[1.00153898e+00],
       [9.98546324e-01],
       [4.27509402e-05]])

In [318]:
q1_reprojected = Pi(P1 @ optimised_Q)
q2_reprojected = Pi(P2 @ optimised_Q)
print(q1, q1_reprojected)
print(q2, q2_reprojected)
diff1 = np.linalg.norm(q1_reprojected - q1_noise)
diff2 = np.linalg.norm(q2_reprojected - q2_noise)
diff1, diff2, np.linalg.norm(Pi(optimised_Q) - Q)

[[1300.]
 [1100.]] [[1301.04731247]
 [1098.95254574]]
[[635.]
 [435.]] [[635.05378922]
 [434.94904663]]


(0.06701027107669148, 1.3401508667857065, 0.00211741543362531)